The following code constructs the following set consisting of all elliptic curves in the Cremona database:
$$X=\{(a_1,a_2,a_3,a_4,a_6), E(\mathbb{Q})_{\text{tors}}, (rmm(E)) |\text{ for }E:y^2+a_1 x + a_3 y = x^3 + a_2 x^2 + a_4 x + a_6\text{ an elliptic curve in Cremona's database}\}$$

In [1]:
c = CremonaDatabase('cremona')
X=[]
for n in (11..499998):
	if len(c.isogeny_classes(n))>0:
		m=len(c.isogeny_classes(n))
		for l in (0..m-1):
			k=len(c.isogeny_classes(n)[l])
			for u in (0..k-1):
				v=c.isogeny_classes(n)[l][u][0]
				E=EllipticCurve(v).minimal_model()
				X.append([v,E.torsion_subgroup().invariants(),(E.a1(),E.a2(),E.a3())])

Next, we create empty sets $T[n]$, $S[n]$, and $Q[n][m]$ where $n$ corresponds to one of the fifteen torsion subgroups and $m$ ranges from $1$ through $12$. More precisely, we let the torsion subgroup $C_n$ correspond to $n$ and the torsion subgroup $C_2 \times C_{2k}$ correspond to $2k$. Below, we identify $n$ with its corresponding torsion subgroup, and say that the torsion subgroup is of type $n$.

We also construct the sets $R[m]=R_i,$ where $R_i=(a_1,a_2,a_3)$ is one of the twelve possibilities occuring in (1.1).

In [2]:
T = [ [] for n in (0..28)]
S = [ [] for n in (0..28)]
Q = [ [ []  for n in (0..12)    ] for  m in (0..28)  ]
R = [ [], (0,0,0), (0,0,1), (0,-1,0), (0,-1,1), (0,1,0), (0,1,1), (1,0,0), (1,0,1), (1,-1,0), (1,-1,1), (1,1,0), (1,1,1) ]
T[1],T[12],T[22],T[24],T[26],T[28]=(),(12,),(2,2),(2,4),(2,6),(2,8)
for n in (2..10):
	T[n] = (n,)

The code below constructs the set:
$$Q[n][m] = \{ x \in X |\text{ the elliptic curve }E\text{ determined by }x\text{ has torsion subgroup of type }n \text{ and rmm}=R_m\}.$$

In [3]:
for x in X:
	for n in (1..28):
		if T[n] == x[1]:
			S[n].append(x)
			for m in (1..12):
				if R[m] == x[2]:
					Q[n][m].append(x)

The code below verifies the numbers given in Table 4.

In [4]:
for n in (1..28):
	if n in (1..10) or n == 12 or n==22 or n==24 or n==26 or n==28:
		t=len(S[n])
		print([n,t,[100*(QQ(len(Q[n][m])/t)).numerical_approx(digits=3) for m in (1..12)]])

[1, 1683021, [17.0, 5.54, 11.7, 3.63, 11.3, 3.73, 6.85, 6.71, 10.1, 10.1, 6.67, 6.72]]
[2, 1186350, [18.5, 0.000, 14.4, 0.000, 14.3, 0.000, 7.84, 8.10, 10.5, 10.2, 8.11, 7.97]]
[3, 51405, [7.52, 7.67, 0.000, 0.000, 8.79, 9.29, 16.9, 19.7, 14.4, 15.7, 0.000, 0.000]]
[4, 33558, [12.9, 0.000, 15.3, 0.000, 15.7, 0.000, 14.9, 3.89, 2.99, 13.3, 3.94, 17.0]]
[5, 1503, [0.000, 0.000, 0.000, 10.8, 0.000, 16.6, 39.0, 0.000, 0.000, 0.000, 0.000, 33.6]]
[6, 6759, [5.33, 0.000, 0.000, 0.000, 8.73, 0.000, 24.1, 28.4, 15.7, 17.8, 0.000, 0.000]]
[7, 80, [0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 73.8, 0.000, 0.000, 26.3, 0.000, 0.000]]
[8, 178, [0.000, 0.000, 4.49, 0.000, 12.9, 0.000, 59.0, 0.000, 0.000, 0.000, 0.000, 23.6]]
[9, 20, [0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 75.0, 0.000, 0.000, 25.0, 0.000, 0.000]]
[10, 42, [0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 100., 0.000, 0.000, 0.000, 0.000, 0.000]]
[12, 17, [0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 41.2, 23.5, 0.000, 35.3, 0.000, 0.000]]
[22

The following code is the algorithm attained from Proposition 3.3. Note, that the algorithm below is redundant, as it asks for sage to compute the reduced minimal model of $E$ in order to obtain the invariants $c_4$ and $c_6$ associated to a global minimal model of $E$. This makes this implementation redundant. See definitions.sage for a version of the algorithm that takes $c_4$ and $c_6$ of a global minimal model and outputs the reduced minimal model of $E$. 

In [5]:
def RMM(E):
	c4=E.minimal_model().c4()
	c6=E.minimal_model().c6()
	a1=ZZ(mod(c6,2))
	c=ZZ(mod(2^(a1-1)*c6,24))
	if c == 0:
		a2,a3,A,B = 0,0, c4,2*c6
	if c == 12:
		a2,a3,A,B = 0,1,c4,2*(c6+216)
	if c == 8:
		a2,a3,A,B = -1,0,c4-16,2*(-6*c4+c6+32)
	if c == 20:
		a2,a3,A,B = -1,1,c4-16,2*(-6*c4+c6+248)
	if c == 16:
		a2,a3,A,B = 1,0,c4-16,2*(6*c4+c6-32)
	if c == 4:
		a2,a3,A,B = 1,1,c4-16,2*(6*c4+c6+184)
	if c == 23:
		a2,a3,A,B = 0,0,c4-1,3*c4 + 2*c6 -1
	if c == 11:
		a2,a3,A,B = 0,1,c4+23,3*c4 + 2*c6 +431
	if c == 3:
		a2,a3,A,B = -1,0,c4-9,-9*c4 + 2*c6 +27
	if c == 15:
		a2,a3,A,B = -1,1,c4+15,-9*c4 + 2*c6 +459
	if c == 19:
		a2,a3,A,B = 1,0,c4-25,15*c4 + 2*c6 -125
	if c == 7:
		a2,a3,A,B = 1,1,c4-1,15*c4 + 2*c6 + 307
	return [a1,a2,a3,-A/48,-B/1728]

In [6]:
U=[]
for x in X:
	E=EllipticCurve(x[0])
	U.append(RMM(E) == x[0])
print(Set(U))

{True}
